## 7.4

In [1]:
import numpy as np
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

import numpy as np
import dask.array as da
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from dask_ml.decomposition import PCA
from dask_ml.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import pandas as pd
import dask
import dask_memusage
from dask.distributed import Client, LocalCluster


In [2]:
from pymks import (
    plot_microstructures,
    PrimitiveTransformer,
    FlattenTransformer,
    TwoPointCorrelation,
    FlattenTransformer,
 )

In [3]:
    cluster = LocalCluster(n_workers=1, threads_per_worker=1,
                           memory_limit=None)
    

In [4]:
    dask_memusage.install(cluster.scheduler, "memusage.csv")


In [5]:
dask.config.set(scheduler='single-threaded')

In [6]:
#52s
myfile ='/home/berkay/Desktop/ThesisCodes/GraspiRelated/big_frame.csv'
big_frame = pd.read_csv(myfile)

big_frame

,0,1,2,3,4,5,6,7,8,9,...,40491,40492,40493,40494,40495,40496,40497,40498,40499,40500
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1703,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1704,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1705,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1706,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


## 9.2

In [7]:
x_data = big_frame.to_numpy()

In [8]:
x_data.shape

(1708, 40501)

In [9]:
y_data = np.load('/home/berkay/Desktop/ThesisCodes/GraspiRelated/y_data.npy')

In [10]:
y_data.shape

(1708, 1)

In [11]:
data_2pt = x_data.reshape(1708,101,401)
data_2pt.shape

(1708, 101, 401)

## 9.2

In [12]:
pri = PrimitiveTransformer(n_state=2, min_=0.0, max_=1.0).transform(data_2pt).compute()

In [13]:
Sample_data = pri[:600]

## 10.2

In [ ]:
Twopt = TwoPointCorrelation(
         periodic_boundary=False,
         cutoff=data_2pt.shape[1],
         correlations=[[0, 0]]
    ).transform(Sample_data)


memory_usage (MB): 3036.00390625
at before debug
array into fftn
483364800
(600, 201, 501)
float64
dask.array<rechunk-merge, shape=(600, 201, 501), dtype=float64, chunksize=(600, 201, 501), chunktype=numpy.ndarray>
(1, 2)

memory_usage (MB): 2972.5


 
 
 - import pymks dask.dafftn and take it apart
 - Create a random dask array
 - Create a random np array same size
 - Check the memory usage
 
 Check other fft libraries!!
 
    

In [ ]:
600 * 101 * 101 * 64 / 8 / 1024**2

In [ ]:
Twopt.nbytes / 1024**2

went down 10.5

## Max 14.1  10.3

In [ ]:

Twopt

In [ ]:
Twopt.compute()

In [ ]:
pri.nbytes / 1024 / 1024 / 1024

In [ ]:
import sys
sys.getsizeof(pri[0,0,0,0])

In [ ]:
#Compute Non-Periodic 2-Point Statistics for 150 samples
model_ = Pipeline(steps=[
    ('discretize', PrimitiveTransformer(n_state=2, min_=0.0, max_=1.0)),
    ('correlations', TwoPointCorrelation(
        periodic_boundary=False,
        cutoff=data_2pt.shape[1],
        correlations=[[0, 0], [1, 1], [0, 1]]
    ))
])

print(data_2pt.shape)
x_stats = model_.transform(data_2pt).persist()
print(x_stats.shape)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

for i in range(200):
    
    plot_microstructures(x_stats[i, :, :, 0], x_stats[i, :, :, 1], titles=['0 -> 0', '1 -> 1'], cmap='bwr')